In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pytz
from datetime import datetime
import MetaTrader5 as mt5

In [12]:
if not mt5.initialize():
	print(f"initialize() failed, error code = {mt5.last_error()}")
	mt5.shutdown()

In [ ]:
def trade_request(symbol: str, direction: str, volume: float, sl_dist: float, tp_dist: float):
    """
    Create a trade request for MetaTrader 5.

    Parameters
    ----------
    symbol    : str -- Trading symbol (e.g., "EURUSD").
    direction : str -- Trade direction, only 'buy' or 'sell'.
    volume  : float -- Trade volume.
    sl_dist : float -- Stop-loss distance from the entry price.
    tp_dist : float -- Take-profit distance from the entry price.

    Returns
    -------
    dict -- Dictionary ready to be sent to MetaTrader 5 containing the trade specifications.
    """

    tick = mt5.symbol_info_tick(symbol)
    if tick is None: raise ValueError(f"Symbol {symbol} not found")


    if direction == "buy":
        order_type  = mt5.ORDER_TYPE_BUY
        price       = tick.ask
        stop_loss   = price - sl_dist
        take_profit = price + tp_dist

    elif direction == "sell":
        order_type  = mt5.ORDER_TYPE_SELL
        price       = tick.bid
        stop_loss   = price + sl_dist
        take_profit = price - tp_dist

    else: raise ValueError("direction must be 'buy' or 'sell'")

    request = {
        "action"		: mt5.TRADE_ACTION_DEAL,
        "symbol"		: symbol,
        "volume"		: volume,
        "type"  		: order_type,
        "price" 		: price,
        "sl"    		: stop_loss,
        "tp"    		: take_profit,
        "deviation" 	: 20,
        "magic"     	: 123456,
        "comment"   	: f"{direction} {symbol}",
        "type_time"    	: mt5.ORDER_TIME_GTC,
        "type_filling" 	: mt5.ORDER_FILLING_FOK,
    }

    return request


In [ ]:
request = trade_request(symbol='EURUSD', direction='buy', volume=0.01, sl_dist=0.0020, tp_dist=0.0060)
result  = mt5.order_send(request)
print(result)

In [ ]:
if result.retcode == mt5.TRADE_RETCODE_DONE:
    print("Trade executed successfully")
else:
    print("Trade failed:", result.retcode)
